In [1]:
import ray, random, os 
ray.shutdown()# Set the random seed for reproducibility
random.seed(42)

#

In [2]:
import ray, random, os, sys
from ray.air.config import ScalingConfig
import pandas as pd
import numpy as np
# ray.init(logging_level="ERROR",ignore_reinit_error=True, _temp_dir='/Users/floriankockler/raytemp/')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from worker_standlone import WorkerStandAlone
from multi_agent import MultiAgent
from manager import Manager
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.rllib.utils import check_env
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.evaluation.episode import Episode
from ray.rllib.evaluation import RolloutWorker
from ray.tune.logger import pretty_print
from ray.tune.logger import TBXLoggerCallback
from typing import Dict


2023-10-17 21:33:20,745	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [3]:
from ray.rllib.utils import check_env
check_env(MultiAgent())

In [4]:

multi_agent_env = MultiAgent()

# Initialize environment
obs, info = multi_agent_env.reset()

# Loop for a maximum of 6310 steps or until done
for step in range(5):
    actions = {}
    
    # Collect sample actions for each agent
    for agent_id, agent_obs in obs.items():
        action_space = multi_agent_env.action_space[agent_id]
        action = action_space.sample()
        actions[agent_id] = action
    
    # Step the environment
    obs, reward, done, _, info = multi_agent_env.step(actions)
    
    # Check if any agent is done and reset the environment if so
    if done["__all__"]:
        obs, info = multi_agent_env.reset()

In [5]:
multi_agent_env = MultiAgent()


for episode in range(0):
    obs, info = multi_agent_env.reset()
    done = {"__all__": False}
    
    print(f"Episode {episode + 1}")

    while not done["__all__"]:
        actions = {}
        
        # Collect actions for each agent
        for agent_id, agent_obs in obs.items():
            action_space = multi_agent_env.action_space[agent_id]
            action = action_space.sample()
            actions[agent_id] = action

        # Step the environment
        obs, reward, done, _, info = multi_agent_env.step(actions)

        # print(f"Actions: {actions}")
        # print(f"Observations: {obs}")
        # print(f"Rewards: {reward}")
        # print(f"Done flags: {done}")
        # print(f"Info: {info}")

    print("Episode done!")

In [17]:

import ray
from ray import air, tune
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import Episode, RolloutWorker
from ray.rllib.policy import Policy
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.algorithms.pg.pg import PGConfig




class MyCallbacks5(DefaultCallbacks):

    def setup(self, **info):
        pass  # Add any setup logic here if needed
    def get_state(self, **info):
        pass  # Add any setup logic here if needed

    def on_step_begin(self, **info):
        # print("info: ",info)
        # info['result'].setdefault('custom_metrics', {})
        # info['result']['custom_metrics']['performance'] = 10
        # print("Callback5 step begin")
        pass  # Add logic here if needed
    def on_trial_save(self, **info):
        pass  # Add logic here if needed
    
    def on_checkpoint(self, **info):
        pass  # Add logic here if needed
    
    def on_step_end(self, **info):
        print("info step end",info)
        # print("Callback5 step end")
        pass  # Add logic here if needed
    def on_trial_start(self, **info):
        print("Callback5 ontrial start")
        pass  # Add logic here if needed
    def on_trial_result(self, **info):
        print("Callback5 ontrial result")
        pass  # Add logic here if needed
    
    def on_experiment_end(self, **info):
        pass  # Add logic here if needed
    
    def on_episode_start(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs
    ):
        # Make sure this episode has just been started (only initial obs
        # logged so far).
        print("Callback5 episode start")
        assert episode.length == 0, (
            "ERROR: `on_episode_start()` callback should be called right "
            "after env reset!"
        )
        # Create lists to store angles in
        episode.user_data["pole_angles"] = []
        episode.hist_data["pole_angles"] = []

    def on_episode_step(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs
    ):
        print("callback5 on episode step")
        # Make sure this episode is ongoing.
        # assert episode.length > 0, (
        #     "ERROR: `on_episode_step()` callback should not be called right "
        #     "after env reset!"
        # )
        # pole_angle = abs(episode.last_observation_for()[2])
        # raw_angle = abs(episode.last_raw_obs_for()[2])
        # assert pole_angle == raw_angle
        # episode.user_data["pole_angles"].append(pole_angle)

        # # Sometimes our pole is moving fast. We can look at the latest velocity
        # # estimate from our environment and log high velocities.
        # if np.abs(episode.last_info_for()["pole_angle_vel"]) > 0.25:
        #     print("This is a fast pole!")

    def on_episode_end(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: Episode,
        env_index: int,
        **kwargs
    ):
        print("callback5 on episode end")
        # Check if there are multiple episodes in a batch, i.e.
        # "batch_mode": "truncate_episodes".
        # if worker.config.batch_mode == "truncate_episodes":
        #     # Make sure this episode is really done.
        #     assert episode.batch_builder.policy_collectors["default_policy"].batches[
        #         -1
        #     ]["dones"][-1], (
        #         "ERROR: `on_episode_end()` should only be called "
        #         "after episode is done!"
        #     )
        # pole_angle = np.mean(episode.user_data["pole_angles"])
        # episode.custom_metrics["pole_angle"] = pole_angle
        # episode.hist_data["pole_angles"] = episode.user_data["pole_angles"]

In [18]:

if os.name == 'nt':
    path_to_save = "C:\\GitHub\\ray\\workertune"
else:
    # path_to_save = "/Volumes/ssdmac/ray/4"
    path_to_save = "/Users/floriankockler/raytemp/3/"

log_file = open("console_logs_ma4.txt", "w")
sys.stdout = log_file

from ray.rllib.env import BaseEnv

from ray.tune.logger import LoggerCallback


class TestLoggerCallback(LoggerCallback):
    def on_trial_result(self, iteration, trials, trial, result, **info):
        print(f"TestLogger for trial {trial}: {result}")

class MyCustomCallback(TBXLoggerCallback):
    def on_episode_end(self, *, worker, episode, **kwargs):
        # portfolio_return = episode.last_info_for()['portfolio_return']
        portfolio_return = episode.last_info_for()
        print("LOGGING of INFO:", portfolio_return)
        episode.custom_metrics["portfolio_return"] = portfolio_return

class MyCustomCallback2(TBXLoggerCallback):
    def on_episode_step(self, *, worker, episode, **kwargs):
        agent_id = "controller"  # Replace with the actual agent ID
        portfolio_return = episode.last_info_for(agent_id).get('portfolio_return', 0)
        print("custom Logger:",portfolio_return)
        episode.custom_metrics[f"{agent_id}_portfolio_return"] = portfolio_return

class MyCallback3(DefaultCallbacks):
    def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: Episode,
                       **kwargs):      
        print("INSIDE CALLBACK: ", episode._agent_to_last_info)


env = MultiAgent()
def env_creator(env_config):
    return MultiAgent()  

register_env("MultiAgent", env_creator)

asha_scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=300,
    grace_period=50,
    reduction_factor=3,
    brackets=1,
)

def create_policy_spec(agent_id):
    print(f"Creating policy for {agent_id}")
    return PolicySpec(
        observation_space=env.observation_space[agent_id],
        action_space=env.action_space[agent_id],
        config={}
    )

controller_policy_spec = PolicySpec(
    observation_space=env.observation_space['controller'],
    action_space=env.action_space['controller'],
    config={}
)

policies = {
    "controller_policy": controller_policy_spec,
}

for agent_id in env.agents:
    policies[agent_id] = create_policy_spec(agent_id)

class MyCallback4(DefaultCallbacks):
    def on_episode_end(self, *, worker, base_env, policies, episode, **kwargs):
        print("INSIDE CALLBACK: ", episode._agent_to_last_info)

def policy_mapping_fn(agent_id, episode=None, agent=None, **kwargs):
    if agent_id == 'controller':
        # print(f"!!!!!! policy mapping manager: {agent_id}")
        return "controller_policy"
    elif agent_id in env.agents:
        return agent_id
    else:
        print("defaul policy triggered")
        return "default_policy"

param_space = {
     "env": "MultiAgent",
    "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "rollout_fragment_length": "auto",
        "framework": "tf",
        "lr": tune.uniform(1e-5,1e-4),
        "gamma": tune.uniform(0.95, 0.9999),
        "lambda": tune.uniform(0.9,1.0),
        "entropy_coeff": tune.uniform(0.01,0.1),
        "vf_loss_coeff": tune.uniform(0.1,0.3),
        "num_workers": 3, 
        # Change for Debugging
        "log_level": "ERROR",
        "output": "logdir",
        "monitor": True,
}

analysis = tune.run(
    "A2C", 
    metric="episode_reward_mean",
    # metric="portfolio_return",
    num_samples=10,
    resume=False,
    callbacks=[MyCallbacks5()],
    mode="max",
    config=param_space, 
    local_dir=path_to_save,
    search_alg=None,
    scheduler=asha_scheduler,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=300),
    max_concurrent_trials=1,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 1,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
    )

print(pretty_print(analysis.last_result))
print("Best hyperparameters found were: ", pretty_print(analysis.best_config))

2023-10-17 21:36:40,974	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
(pid=31376) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=31376) 2023-10-17 21:36:48,784	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=31376) 2023-10-17 21:36:48,784	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
A2C_MultiAgent_7ede5_00000,180096,"{'ObsPreprocessorConnector_ms': 0.02167622248331706, 'StateBufferConnector_ms': 0.0023265679677327475, 'ViewRequirementAgentConnector_ms': 0.06289084752400716}","{'num_env_steps_sampled': 45024, 'num_env_steps_trained': 45024, 'num_agent_steps_sampled': 180096, 'num_agent_steps_trained': 180096}",{},6305,{},0,0,0,1,"{'learner': {'controller_policy': {'learner_stats': {'cur_lr': 1.7546763046993874e-05, 'entropy_coeff': 0.014056925661861897, 'policy_loss': -0.024278514, 'policy_entropy': 812.9315, 'var_gnorm': 23.198652, 'vf_loss': 2.4091836e-08, 'model': {}, 'grad_gnorm': 12.613525, 'vf_explained_var': -1.0}}, 'ABT.US': {'learner_stats': {'cur_lr': 1.7546763046993874e-05, 'entropy_coeff': 0.014056925661861897, 'policy_loss': -0.0006072004, 'policy_entropy': 301.7782, 'var_gnorm': 23.038738, 'vf_loss': 1.573667e-09, 'model': {}, 'grad_gnorm': 7.2092333, 'vf_explained_var': -1.0}}, 'MRK.US': {'learner_stats': {'cur_lr': 1.7546763046993874e-05, 'entropy_coeff': 0.014056925661861897, 'policy_loss': 0.0032201838, 'policy_entropy': 302.3126, 'var_gnorm': 22.996954, 'vf_loss': 6.022748e-09, 'model': {}, 'grad_gnorm': 7.189658, 'vf_explained_var': 0.5856692}}, 'PFE.US': {'learner_stats': {'cur_lr': 1.7546763046993874e-05, 'entropy_coeff': 0.014056925661861897, 'policy_loss': -0.0006262958, 'policy_entropy': 299.7364, 'var_gnorm': 23.00664, 'vf_loss': 2.137931e-10, 'model': {}, 'grad_gnorm': 7.2020454, 'vf_explained_var': -1.0}}}, 'num_env_steps_sampled': 45024, 'num_env_steps_trained': 45024, 'num_agent_steps_sampled': 180096, 'num_agent_steps_trained': 180096}",180096,180096,45024,5472,541.989,45024,5472,541.989,0,3,0,0,5472,"{'cpu_util_percent': 58.850000000000016, 'ram_util_percent': 84.36428571428573}","{'controller_policy': 0.0, 'ABT.US': 0.0, 'MRK.US': 0.0, 'PFE.US': 0.0}","{'controller_policy': 0.0, 'ABT.US': 0.0, 'MRK.US': 0.0, 'PFE.US': 0.0}","{'controller_policy': 0.0, 'ABT.US': 0.0, 'MRK.US': 0.0, 'PFE.US': 0.0}","{'mean_raw_obs_processing_ms': 0.9542613985494288, 'mean_inference_ms': 1.845284655805015, 'mean_action_processing_ms': 0.5604967839826883, 'mean_env_wait_ms': 0.8373553436233001, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 0.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.0, 'episode_len_mean': 6305.0, 'episode_media': {}, 'episodes_this_iter': 1, 'policy_reward_min': {'controller_policy': 0.0, 'ABT.US': 0.0, 'MRK.US': 0.0, 'PFE.US': 0.0}, 'policy_reward_max': {'controller_policy': 0.0, 'ABT.US': 0.0, 'MRK.US': 0.0, 'PFE.US': 0.0}, 'policy_reward_mean': {'controller_policy': 0.0, 'ABT.US': 0.0, 'MRK.US': 0.0, 'PFE.US': 0.0}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'episode_lengths': [6305, 6305, 6305, 6305, 6305, 6305], 'policy_controller_policy_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'policy_ABT.US_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'policy_MRK.US_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'policy_PFE.US_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.9542613985494288, 'mean_inference_ms': 1.845284655805015, 'mean_action_processing_ms': 0.5604967839826883, 'mean_env_wait_ms': 0.8373553436233001, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.02167622248331706, 'StateBufferConnecto

In [ ]:
env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)



analysis = tune.run(
        "A2C",
        metric="episode_reward_mean",
        mode="max",
        config={
            "env": "worker",
            "env_config": {"initial_capital": 1e6},
            "lr": tune.uniform(1e-5, 1e-4),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
}, )

In [ ]:


n_iterations = 30000


env = WorkerStandAlone(data=data)

state = env.reset()

for _ in range(n_iterations):
    action = env.action_space.sample()  # Generate random actions for the manager
    worker_dones = {...}  # You need to provide the values for worker_dones
    worker_truncateds = {...}  # You need to provide the values for worker_truncateds

    obs, reward, done, truncated, info = env.step(action)

    if done:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:

env = HRL()

n_iterations = 1

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 



policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}

    
algo = PPOConfig().environment(env=HRL).multi_agent(
        policies=policies,
        policy_mapping_fn = policy_mapping_fn,  
    ).training(train_batch_size=4000).build()

print(algo.train())

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

analysis = tune.run("A2C", metric="episode_reward_mean", mode="max",config=param_space)

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
algo = PPOConfig().environment(env=HRL).multi_agent(
    policies={
        "policy_1": ()
    }
)

In [ ]:
from ray.tune import CLIReporter
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
env = HRL()

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune_config,
    param_space=param_space,

    run_config=run_config,
)
results = tuner.fit()


In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()